In [1]:
import sqlite3

In [2]:
utable = """ CREATE TABLE User
(
    id NUMBER(50),
    name VARCHAR(50),
    screen_name VARCHAR(50),
    description VARCHAR(300),
    friends_count NUMBER(20),
    CONSTRAINT ID_PK,
        PRIMARY KEY(id)
);
"""

In [3]:
#Modify your SQL table from Module 5 to include “user_id” columns 
#which will be a foreign key referencing the user table.
table = """ CREATE TABLE Tweets
(
    created_at DATE,
    id_str NUMBER(50),
    text VARCHAR(300),
    source VARCHAR(200),
    in_reply_to_user_id VARCHAR(50),
    in_reply_to_screen_name VARCHAR(50),
    in_reply_to_status_id VARCHAR(50),
    retweet_count NUMBER(30),
    contributors VARCHAR(50),
    user_id NUMBER(50),
    CONSTRAINT TweetPK
        PRIMARY KEY(id_str),
    CONSTRAINT UserFK
        FOREIGN KEY (user_id) REFERENCES User(id)
);
""" 

In [4]:
conn = sqlite3.connect("dsc450.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS User")
cursor.execute("DROP TABLE IF EXISTS Tweets")

In [5]:
cursor.execute(table)
cursor.execute(utable)

In [11]:
import json
import urllib.request as urllib
import time

In [18]:
webFile = urllib.urlopen("http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/Assignment5.txt")

In [19]:
#some of the tweets are intentionally damaged 
#and will not parse properly. You will need to store these tweets in a separate “error” file
allTweets = webFile.readlines()
inFile = open("error.txt","w")
for tweet in allTweets:
    try:
        tdict = json.loads(tweet.decode("utf8"))
        cursor.execute("INSERT OR IGNORE INTO Tweets VALUES (?,?,?,?,?,?,?,?,?,?);",
                      (tdict['created_at'],
                       tdict['id_str'],
                       tdict['text'],
                       tdict['source'],
                       tdict['in_reply_to_user_id'],
                       tdict['in_reply_to_screen_name'],
                       tdict['in_reply_to_status_id'],
                       tdict['retweet_count'],
                       tdict['contributors'],
                       tdict['user']['id']))
        cursor.execute("INSERT OR IGNORE INTO User VALUES (?,?,?,?,?);",
                      (tdict['user']['id'],
                       tdict['user']['name'],
                       tdict['user']['screen_name'],
                       tdict['user']['description'],
                       tdict['user']['friends_count']))
    except ValueError:
        inFile.write(str(tweet))

In [20]:
userTable = cursor.execute("SELECT * FROM User Limit 5; ").fetchall()
tweetTable = cursor.execute("SELECT * FROM Tweets Limit 5; ").fetchall()

In [21]:
print("SELECT * FROM User LIMIT 5; ")
for r in userTable:
    print(str(r)+'\n')

SELECT * FROM User LIMIT 5; 
(367361405, 'gugusuarez', 'guadalupesuare4', None, 543)

(1605442621, 'ほたる', 'htr_ruby', '黒バス/青エク/あの花/君僕/進撃/Free! etc.只今黒バス/ハイキュー!!が熱いです 詳しくはツイプロで(ฅ•ω•ฅ)', 117)

(1685288690, '電話占いピュアリ', 'purely_c', '人気・口コミランキングNo.1！当たると評判の電話占い。タレントや芸能人も御用達でテレビや雑誌で話題の一流占い師が多数在籍。復縁や複雑な恋愛相談はお任せ下さい。初回最大4000円分無料！', 0)

(766497559, 'Veronica', 'verodoglover', None, 89)

(229690290, 'こさんつ', 'Cosan2', '(ﾟωﾟ)\u3000東方やってます。 たまにお絵かき。\u3000天則のことつぶやくことが多いかも\u3000 規制用垢(@Cosantwo) 絵はじめました(24.7/4)', 610)



In [22]:
print("SELECT * FROM Tweets Limit 5; ")
for r in tweetTable:
    print(str(r)+'\n')

SELECT * FROM Tweets Limit 5; 
('Tue May 20 00:00:19 +0000 2014', 468541694288207874, 'la asusto a selena me dice es joda te vy a extrañar jajajajaja ni m fui pero ta vy a tener tiempo libre y todo wi', '<a href="https://mobile.twitter.com" rel="nofollow">Mobile Web (M2)</a>', None, None, None, 0, None, 367361405)

('Tue May 20 00:00:19 +0000 2014', 468541694284017664, '【現在視聴中のアニメ】ソウルイーターノット!/僕らはみんな河合荘/メカクシティアクターズ/一週間フレンズ。/悪魔のリドル/ハイキュー!! etc…', '<a href="http://makebot.sh" rel="nofollow">ほたるボット</a>', None, None, None, 0, None, 1605442621)

('Tue May 20 00:00:19 +0000 2014', 468541694279835649, '瑤潤 嬉陽(ようじゅん きひ)先生が待機開始しました', '<a href="http://admin.pure-c.jp/prog01_test3/tweet_app/twitter_sample.php" rel="nofollow">purely_c tweet app</a>', None, None, None, 0, None, 1685288690)

('Tue May 20 00:00:19 +0000 2014', 468541694292402176, '10 More FREE Pampers Gifts to Grow Points! via http://t.co/3Sgq2UVq67 - We have a new Pampers Gifts ... http://t.co/DJa59HyRDi', '<a href="http://www.feedbli

In [24]:
inFile.close()
conn.commit()
conn.close()